## Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

## 1. Download and Explore Dataset

In [1]:
# import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Folium for mapping
import folium # map rendering library

# K-means clustering
from sklearn.cluster import KMeans

print('Libraries imported.')

/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Libraries imported.


### 1.1. Load data from Wiki

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

tables = pd.read_html(r.text)
tables

[               0                 1  \
 0    Postal code           Borough   
 1            M1A      Not assigned   
 2            M2A      Not assigned   
 3            M3A        North York   
 4            M4A        North York   
 5            M5A  Downtown Toronto   
 6            M6A        North York   
 7            M7A  Downtown Toronto   
 8            M8A      Not assigned   
 9            M9A         Etobicoke   
 10           M1B       Scarborough   
 11           M2B      Not assigned   
 12           M3B        North York   
 13           M4B         East York   
 14           M5B  Downtown Toronto   
 15           M6B        North York   
 16           M7B      Not assigned   
 17           M8B      Not assigned   
 18           M9B         Etobicoke   
 19           M1C       Scarborough   
 20           M2C      Not assigned   
 21           M3C        North York   
 22           M4C         East York   
 23           M5C  Downtown Toronto   
 24           M6C        

### 1.2. Clean data

In [3]:
df=pd.DataFrame(tables[0])

#1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df.columns=['Postcode','Borough','Neighbourhood']

df.drop([0],axis=0,inplace=True)

df.reset_index()

#2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

#3. More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, 
# you will notice that M5A is listed twice and has two neighborhoods: 
# Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods 
# separated with a comma as shown in row 11 in the above table.

df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))

#4. If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, 
# the value of the Borough and the Neighborhood columns will be Queen's Park.

df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']

df1.reset_index()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [4]:
df1.reset_index().shape

(103, 3)

### 1.3. Get geo data

In [5]:
#get geo data from local drive (saved from link https://cocl.us/Geospatial_data)
geo_data=pd.read_csv("Geospatial_Coordinates.csv")
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### 1.4. Integrating postal

In [6]:
df1['Latitude']=geo_data['Latitude'].values
df1['Longitude']=geo_data['Longitude'].values

df2 = df1.reset_index()
df2

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


In [7]:
df2.shape

(103, 5)

### 1.5. Visualizing

In [8]:
center_lat=df1.Latitude.mean()
center_long=df1.Longitude.mean()

toronto_map = folium.Map(location=[center_lat, center_long], zoom_start=13) # generate map centred around the Conrad Hotel

dff=df2
for lat, lng, borough, neighborhood in zip(dff.Latitude, dff.Longitude, dff.Borough, dff.Neighbourhood):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.4
    ).add_to(toronto_map)
toronto_map

## 2. Explore Neighborhoods in City

### 2.1. Borrow the pre-processing  function from foursquare lab

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
boroughs = set(dff['Borough'])
toronto_boroughs = {borough for borough in boroughs if 'Scarborough' in borough}
toronto_data = dff[dff['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


### 2.2. Apply Foursquare API credentials and get places data

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [41]:
toronto_venues = getNearbyVenues(toronto_data.Neighbourhood, toronto_data.Latitude, toronto_data.Longitude)
toronto_venues

,Neighborhood,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,Guildwood / Morningside / West Hill,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
7,Guildwood / Morningside / West Hill,43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,Guildwood / Morningside / West Hill,43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,Guildwood / Morningside / West Hill,43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [13]:
toronto_venues.groupby('Neighborhood').count()

,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Birch Cliff / Cliffside West,4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
Clarks Corners / Tam O'Shanter / Sullivan,13,13,13,13,13,13
Cliffside / Cliffcrest / Scarborough Village West,3,3,3,3,3,3
Dorset Park / Wexford Heights / Scarborough Town Centre,7,7,7,7,7,7
Golden Mile / Clairlea / Oakridge,9,9,9,9,9,9
Guildwood / Morningside / West Hill,7,7,7,7,7,7
Kennedy Park / Ionview / East Birchmount Park,6,6,6,6,6,6


### 2.3. Normalize and group data

In [34]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.25,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000
1,Birch Cliff / Cliffside West,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.111111,0.111111,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.00,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.111111,0.000000
3,Clarks Corners / Tam O'Shanter / Sullivan,0.000000,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.076923,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.076923,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.076923,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.076923,0.000000,0.000000,0.153846,0.153846,0.000000,0.000000,0.00,0.00,0.000000,0.076923,0.000000
4,Cliffside / Cliffcrest / Scarborough Village West,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
5,Dorset Park / Wexford Heights / Scarborough To...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.285714,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.142857
6,Golden Mile / Clairlea / Oakridge,0.000000,0.000000,0.222222,0.000000,0.0,0.000000,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.111111,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000000
7,Guildwood / Morningside / West Hill,0.000000,0.000000,0.000000,0.142857,0.

In [15]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1               Skating Rink  0.25
2  Latin American Restaurant  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff / Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4                  Motel  0.00


----Cedarbrae----
                venue  freq
0    Hakka Restaurant  0.11
1              Bakery  0.11
2                Bank  0.11
3     Thai Restaurant  0.11
4  Athletics & Sports  0.11


----Clarks Corners / Tam O'Shanter / Sullivan----
                 venue  freq
0          Pizza Place  0.15
1             Pharmacy  0.15
2    Convenience Store  0.08
3         Noodle House  0.08
4  Fried Chicken Joint  0.08


----Cliffside / Cliffcrest / Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1              

## 3. Analyze Each Neighborhood

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =toronto_grouped['Neighborhood']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
1,Birch Cliff / Cliffside West,General Entertainment,Skating Rink,Café,College Stadium,Coffee Shop,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Bank,Athletics & Sports,Bakery,Bubble Tea Shop
3,Clarks Corners / Tam O'Shanter / Sullivan,Pharmacy,Pizza Place,Chinese Restaurant,Gas Station,Italian Restaurant,Noodle House,Convenience Store,Fast Food Restaurant,Fried Chicken Joint,Bank
4,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Intersection,Motel,Coffee Shop,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


## 4. Cluster Neighborhoods

In [22]:
#Apply K-clustering algorithm
from sklearn.cluster import KMeans
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering

,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.25,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000
2,0.000000,0.111111,0.111111,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.00,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.111111,0.000000
3,0.000000,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.076923,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.076923,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.076923,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.076923,0.000000,0.000000,0.153846,0.153846,0.000000,0.000000,0.00,0.00,0.000000,0.076923,0.000000
4,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.285714,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.142857
6,0.000000,0.000000,0.222222,0.000000,0.0,0.000000,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.111111,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.142857,0.0,0.142857,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.00,0.000000,0.142857,0.142857,0.00,0.00

In [24]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 4, 0, 3, 1, 1, 4], dtype=int32)

In [33]:
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


In [40]:
toronto_merged = toronto_data[0:16]
toronto_merged
toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_)
#toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head(100)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,History Museum,Bar,Vietnamese Restaurant,College Stadium,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,1,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,Bank,Rental Car Location,Breakfast Spot,Electronics Store,Intersection,Mexican Restaurant,Medical Center,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store
3,1,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Pharmacy,Korean Restaurant,Vietnamese Restaurant,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Bank,Athletics & Sports,Bakery,Bubble Tea Shop
5,1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,1,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,Coffee Shop,Hobby Shop,Discount Store,Department Store,Convenience Store,Chinese Restaurant,College Stadium,Gas Station,Gaming Cafe,Furniture / Home Store
7,1,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,Bakery,Bus Line,Soccer Field,Ice Cream Shop,Park,Bus Station,Intersection,Vietnamese Restaurant,College Stadium,Furniture / Home Store
8,1,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476,American Restaurant,Intersection,Motel,Coffee Shop,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,2,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848,General Entertainment,Skating Rink,Café,College Stadium,Coffee Shop,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


## The below cell for reference only

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

### 2.4. Visualize the clusters

In [42]:
latitude = 43.653963
longitude = -79.387207

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [43]:
toronto_merged[toronto_merged['Cluster Labels']==0].head(10)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,0,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849,Middle Eastern Restaurant,Smoke Shop,Bakery,Breakfast Spot,Coffee Shop,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [44]:
toronto_merged[toronto_merged['Cluster Labels']==1]

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,History Museum,Bar,Vietnamese Restaurant,College Stadium,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,1,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,Bank,Rental Car Location,Breakfast Spot,Electronics Store,Intersection,Mexican Restaurant,Medical Center,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store
3,1,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Pharmacy,Korean Restaurant,Vietnamese Restaurant,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Bank,Athletics & Sports,Bakery,Bubble Tea Shop
5,1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,1,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,Coffee Shop,Hobby Shop,Discount Store,Department Store,Convenience Store,Chinese Restaurant,College Stadium,Gas Station,Gaming Cafe,Furniture / Home Store
7,1,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,Bakery,Bus Line,Soccer Field,Ice Cream Shop,Park,Bus Station,Intersection,Vietnamese Restaurant,College Stadium,Furniture / Home Store
8,1,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476,American Restaurant,Intersection,Motel,Coffee Shop,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
13,1,M1T,Scarborough,Clarks Corners / Tam O'Shanter / Sullivan,43.781638,-79.304302,Pharmacy,Pizza Place,Chinese Restaurant,Gas Station,Italian Restaurant,Noodle House,Convenience Store,Fast Food Restaurant,Fried Chicken Joint,Bank


In [45]:
toronto_merged[toronto_merged['Cluster Labels']==2]

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,2,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848,General Entertainment,Skating Rink,Café,College Stadium,Coffee Shop,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


In [46]:
toronto_merged[toronto_merged['Cluster Labels']==3]

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,3,M1S,Scarborough,Agincourt,43.7942,-79.262029,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


In [47]:
toronto_merged[toronto_merged['Cluster Labels']==4]

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,4,M1P,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,43.757410,-79.273304,Indian Restaurant,Vietnamese Restaurant,Pet Store,Gaming Cafe,Furniture / Home Store,Chinese Restaurant,Breakfast Spot,Bubble Tea Shop,Athletics & Sports,Bakery
15,4,M1W,Scarborough,Steeles West / L'Amoreaux West,43.799525,-79.318389,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Grocery Store,Pizza Place,Discount Store,Bubble Tea Shop,Pharmacy,Breakfast Spot,Bank
